In [3]:
import numpy as np
import torch
from torch import nn, optim
import random
from collections import Counter

In [4]:
# 导入文件
with open(file_path) as f:
    text = f.read())

SyntaxError: unmatched ')' (2257673130.py, line 2)

In [5]:
# 预处理
def preprocess(text, freq):
    text = text.lower()
    text.text.replace(".", "<PERIO>")
    #... 
    words = text.split()
    word_counts = Counter(words) #[]
    trimmed_words = [word for word in words if word_counts[word]>freq]
    return trimmed_words


In [6]:
# 准备工作: 词典，文本转为数，训练样本准备

In [7]:
words = preprocess(text)
vocab = set(words)
vocab2int = {w: c for c, w in enumerate(list(vocab))}
int2vocab = {c: w for c, w in enumerate(list(vocab))}

int_words = [vocab2int[w] for w in words]
int_word_counts = Counter(int_words)
total_count = len(int_words)
word_freqs = {w: c/total_count for w,c in int_word_counts.items()}
prob_drop = {w: 1-np.sqrt(t/word_freqs[w]) for w in int_word_counts}
train_words = [w for win int_words if random.random()<(1-prob_drop[w])]

SyntaxError: invalid syntax (380812899.py, line 11)

In [8]:
# 获取周边词/target
def get_target(words, idx, window_size):
    target_window = np.random.randint(1, window_size+1)
    start_int = idx-target_window if (idx-target_window) > 0 else 0
    end_point = idx+target_window
    targets = set(words[start_point:idx]+words[idx+1:endpoint+1])
    return list(targets)

In [9]:
# batch 迭代器
def get_batch(words, batch_size, window_size):
    n_batches = len(words)//batch_size
    words = words[:n_batches*batch_size]
    for idx in range(0, len(words), batch_size):
        batch_x, batch_y = [], []
        batch = words[idx:idx+batch_size]
        for in in range(len(batch)):
            x = batch[i]
            y = get_target(batch, i, window_size)
            batch_x.extend([x]*len(y))
            batch_y.extend(y)
        yield batch_x, batch_y

SyntaxError: invalid syntax (3778667883.py, line 8)

In [10]:
# 构造网络结构
class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noice_dist = noise_dist
        
        # define embedding layers for input and output words
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        # Initialize embedding tables with uniform distribution
        # I believe this helps with convergence
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
    def forward_input(self, input_words):
        input_vectors = self.in_embed(input_words)
        return input_vectors
    
    def forward_output(self, output_words):
        output_vectors = self.out_embed(output_words)
        return output_vectors
    
    def forward_noice(self, batch_size, n_samples):
        if self.noise_dist is None:
            noise_dist = torch.ones(self.n_vocab)
        else:
            noise_dist = self.noise_dist
            
        # Sample words from our noise distribution
        noise_words = torch.multinomial(noise_dist, batch_size*n_samples, replacement=True)
        noise_vectors = self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)

        return noise


In [11]:
# 构造损失函数
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, input_vectors, output_vectors, noise_vectors):
        batch_size, embed_size = input_vectors.shape
        
        # input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        #output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # correct log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        out_loss = out_loss.squeeze()
        
        # incorrect log-sigmoid loss
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)
        
        # degate and sum correct and noiy log-sigoic loss
        # return average batch loss
        return -(out_loss + noise_loss).mean()

In [12]:
# 模型训练
embedding_dim = 300
model = SkipGramNeg(len(vocab2int), embedding_dim)m
noise_dist = noise_dist

# using the loss that we defined
criterion = NegativeSamplingLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 1500
steps = 0
epochs = 5
batch_size = 500
n_samples = 5

# train for some number of epochs 
for e in range(epochs):
    # get our input, target batches
    for input_words, target_words in get_batch(train_words, batch_size):
        steps += 1
        inputs, targets = torch.LongTensor(input_words), torch.LongTensor(target_words)
        # input, output and noise vectors
        input_vectors = model.forward_input(inputs)
        output_vectors = model.forward_output(targets)
        noise_vectors = model.forward_noise(batch_size, n_samples)
        
        # negative sampling loss
        loss = criterion(input_vectors, output_vectors, noise_vectors)
        if steps//print_every = 0:
            print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

SyntaxError: invalid syntax (3005137583.py, line 3)